In [1]:
import numpy as np
import pandas as pd


path = '/opt/ml/input/data/'

train = pd.read_csv(path + 'train_data.csv', parse_dates=["Timestamp"])
test = pd.read_csv(path + 'test_data.csv', parse_dates=["Timestamp"])

dat = pd.concat([train, test], axis = 0)
dat = dat.sort_values(by = ['userID', 'Timestamp'])


In [2]:
diff = dat.loc[:, ["userID", "Timestamp"]].groupby("userID").diff().fillna(pd.Timedelta(seconds=0))
diff['Timestamp'] = diff['Timestamp'].apply(pd.Timedelta.total_seconds)
diff['Timestamp'] = diff['Timestamp'].apply(lambda x: 0 if x >600 or x<0 else x)
dat['solve_time'] = diff['Timestamp']

In [3]:
# 시험 난이도(?)로 추청되는 특성 따로 분류
dat['b_category'] = dat['assessmentItemID'].str[2]
# 시험지로 추청되는 특성 따로 분류. 뒤에 것이 좋을 것 같은데 일단 제출을 한 것으로 써놓음.
dat['test_category'] = dat['assessmentItemID'].str[4:7] # dat['assessmentItemID'].str[2] + dat['assessmentItemID'].str[4:7]
dat['problem_id'] = dat['assessmentItemID'].str[-3:]


In [4]:
diff['Timestamp'] = dat['solve_time']
diff['Timestamp'] = pd.qcut(diff['Timestamp'],5)
diff['Timestamp'] = diff['Timestamp'].astype("str")
dat['category_st_qcut_5'] = diff['Timestamp']

In [5]:
dat['solve_time'] = dat['solve_time'].astype(int)

In [8]:
dat['last_answerCode'] = dat.groupby("userID")['answerCode'].shift(1).fillna(1)\


In [10]:
dat['last_answerCode'] = dat['last_answerCode'].astype(int)

In [12]:
dat['year'] = dat['Timestamp'].dt.year # 연도 정보
dat['month'] =  dat['Timestamp'].dt.month # 월 정보
dat['day'] =  dat['Timestamp'].dt.day # 일 정보
dat['hour'] =  dat['Timestamp'].dt.hour # 시간 정보

In [13]:
dat

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,solve_time,b_category,test_category,problem_id,category_st_qcut_5,last_answerCode,year,month,day,hour
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,0,6,001,001,"(-0.001, 2.0]",1,2020,3,24,0
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,3,6,001,002,"(2.0, 12.0]",1,2020,3,24,0
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,8,6,001,003,"(2.0, 12.0]",1,2020,3,24,0
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,7,6,001,004,"(2.0, 12.0]",1,2020,3,24,0
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,7,6,001,005,"(2.0, 12.0]",1,2020,3,24,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,24,3,071,005,"(12.0, 29.0]",0,2020,6,5,6
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,0,4,165,001,"(-0.001, 2.0]",0,2020,8,21,1
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,11,4,165,002,"(2.0, 12.0]",1,2020,8,21,1
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,46,4,165,003,"(29.0, 66.0]",1,2020,8,21,1


In [7]:
dat.to_csv('/opt/ml/input/data/FE/FE_total.csv', index=False)